In [1]:
from smolagents import LiteLLMModel

model = LiteLLMModel(
    model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")
type(tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast

In [3]:
rendered_prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are an AI assistant with access to various tools."},
    {"role": "user", "content": "Hi !"},
    {"role": "system", "content": "Hi human, what can help you with?"}
], tokenize=False, add_generation_prompt=True)

In [11]:
from typing import Any, Optional
from smolagents.tools import Tool
import duckduckgo_search

class DuckDuckGoSearchTool(Tool):
    name = "web_search"
    description = "Performs a duckduckgo web search based on your query (think a Google search) then returns the top search results."
    inputs = {'query': {'type': 'string', 'description': 'The search query to perform.'}}
    output_type = "string"

    def __init__(self, max_results=10, **kwargs):
        super().__init__()
        self.max_results = max_results
        try:
            from duckduckgo_search import DDGS
        except ImportError as e:
            raise ImportError(
                "You must install package `duckduckgo_search` to run this tool: for instance run `pip install duckduckgo-search`."
            ) from e
        self.ddgs = DDGS(**kwargs)

    def forward(self, query: str) -> str:
        results = self.ddgs.text(query, max_results=self.max_results)
        if len(results) == 0:
            raise Exception("No results found! Try a less restrictive/shorter query.")
        postprocessed_results = [f"[{result['title']}]({result['href']})\n{result['body']}" for result in results]
        return "## Search Results\n\n" + "\n\n".join(postprocessed_results)


web_search = DuckDuckGoSearchTool()

print(len(web_search("what time is it")))


/var/folders/56/_q1k7zs52t9997xq9ncxwdx00000gn/T/ipykernel_33165/979951228.py:20: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


[{'title': 'time - Proper Timezone Acronym Usage - PT vs PDT or PST', 'href': 'https://english.stackexchange.com/questions/88485/proper-timezone-acronym-usage-pt-vs-pdt-or-pst', 'body': 'Nov 16, 2021 · What is the difference between PT (Pacific Time), PDT (Pacific Daylight Time), and PST (Pacific Standard Time)? Also, is the time format "2:00pm PT" unambiguous?'}, {'title': 'What time or which time? - English Language & Usage Stack …', 'href': 'https://english.stackexchange.com/questions/493909/what-time-or-which-time', 'body': 'Apr 15, 2019 · What time tomorrow works best for you? (very open-ended) Going the opposite direction, "what is your favorite hot beverage" would be a different way to frame your first …'}, {'title': 'punctuation for time [colon vs full stop] | WordReference Forums', 'href': 'https://forum.wordreference.com/threads/punctuation-for-time-colon-vs-full-stop.2076421/', 'body': 'Feb 28, 2011 · I wonder if I can use full stop instead of colon when to say time. For exa